I implemented a Random Forest Classifier to detect fraudulent transactions. Random Forest is an ensemble learning method that constructs multiple decision trees during training and combines their outputs to improve accuracy and reduce overfitting. This model is particularly effective on imbalanced datasets like fraud detection, especially when paired with techniques like class weighting.

The data was preprocessed by removing identifiers and categorical columns that did not contribute meaningful information to the prediction. After handling missing values and filtering out extreme outliers, the features were standardized using StandardScaler to ensure consistent scaling across all numerical inputs.

I configured the Random Forest with the following hyperparameters:

n_estimators=200: Uses 200 trees for better generalization.

max_depth=10: Prevents overly complex trees.

min_samples_split=5 and min_samples_leaf=2: Controls tree branching to reduce overfitting.

class_weight='balanced': Automatically adjusts weights inversely proportional to class frequencies, helping the model focus more on detecting rare fraudulent cases.

The model was trained using an 80/20 train-test split with stratification to maintain class proportions. Evaluation was done using confusion matrix, precision, recall, F1-score, and ROC AUC score. Overall, the Random Forest model provided a balanced trade-off between detecting fraud and minimizing false positives.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.impute import SimpleImputer
from statsmodels.stats.outliers_influence import variance_inflation_factor
from collections import Counter
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from xgboost import XGBClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/task/Fraud.csv')

In [ ]:
df.shape

(6362620, 11)

In [ ]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [ ]:
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [ ]:
df.isnull().sum()

,0
step,0
type,0
amount,0
nameOrig,0
oldbalanceOrg,0
newbalanceOrig,0
nameDest,0
oldbalanceDest,0
newbalanceDest,0
isFraud,0


In [ ]:
df = df.dropna()

In [ ]:
numeric_cols = df.select_dtypes(include=np.number).columns
df = df[(np.abs(df[numeric_cols] - df[numeric_cols].mean()) <= (3 * df[numeric_cols].std())).all(axis=1)]

In [ ]:
X_vif = df.select_dtypes(include=np.number).drop(columns=['isFraud'])
vif = pd.DataFrame()
vif["features"] = X_vif.columns
vif["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
vif

/usr/local/lib/python3.11/dist-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


,features,VIF
0,step,1.428556
1,amount,3.782318
2,oldbalanceOrg,402.449844
3,newbalanceOrig,415.412624
4,oldbalanceDest,130.370763
5,newbalanceDest,142.959408
6,isFlaggedFraud,NaN


In [ ]:
X= df.drop(columns=['isFraud'])
y= df['isFraud']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
# Drop non-numeric columns before scaling
X_train = X_train.drop(columns=['type', 'nameOrig', 'nameDest'])
X_test = X_test.drop(columns=['type', 'nameOrig', 'nameDest'])

# Apply StandardScaler to the numerical columns
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
np.unique(y_train)

array([0])

In [ ]:
np.unique(y_train, return_counts=True)

(array([0]), array([4816078]))

In [ ]:
np.unique(df['isFraud'], return_counts=True)

(array([0]), array([6020098]))

In [ ]:
df['isFraud'] = df['isFraud'].map({'No': 0, 'Yes': 1})

In [ ]:
df['isFraud'].value_counts()

,count
isFraud,


In [ ]:
df['isFraud'] = df['isFraud'].map({'No': 0, 'Yes': 1})

In [ ]:
df['isFraud'].unique()

array([nan])

In [ ]:
df.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [ ]:
df = df.rename(columns={'actual_column_name_here': 'is_fraud'})

In [ ]:
df['isFraud'].value_counts()

,count
isFraud,


In [ ]:
df.head(10)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.0,0.00,NaN,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.0,0.00,NaN,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.0,0.00,NaN,0
5,1,PAYMENT,7817.71,C90045638,53860.00,46042.29,M573487274,0.0,0.00,NaN,0
6,1,PAYMENT,7107.77,C154988899,183195.00,176087.23,M408069119,0.0,0.00,NaN,0
7,1,PAYMENT,7861.64,C1912850431,176087.23,168225.59,M633326333,0.0,0.00,NaN,0
8,1,PAYMENT,4024.36,C1265012928,2671.00,0.00,M1176932104,0.0,0.00,NaN,0
9,1,DEBIT,5337.77,C712410124,41720.00,36382.23,C195600860,41898.0,40348.79,NaN,0
10,1,DEBIT,9644.94,C1900366749,4465.00,0.00,C997608398,10845.0,157982.12,NaN,0
11,1,PAYMENT,3099.97,C249177573,20771.00,17671.03,M2096539129,0.0,0.00,NaN,0


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/task/Fraud.csv')

In [ ]:
df['isFraud'].unique()

array([0, 1])

In [ ]:
X = df.drop(columns=['isFraud'])
y = df['isFraud']

In [ ]:
model = XGBClassifier(eval_metric='logloss', use_label_encoder=False)

In [ ]:
model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [ ]:
np.unique(y_train, return_counts=True)

(array([0]), array([4816078]))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


[[1204020]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1204020

    accuracy                           1.00   1204020
   macro avg       1.00      1.00      1.00   1204020
weighted avg       1.00      1.00      1.00   1204020

ROC AUC Score: nan


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))

[[1204020]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1204020

    accuracy                           1.00   1204020
   macro avg       1.00      1.00      1.00   1204020
weighted avg       1.00      1.00      1.00   1204020

ROC AUC Score: nan


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


In [ ]:
importances = pd.Series(model.feature_importances_, index=[f'feature_{i}' for i in range(len(model.feature_importances_))])
print(importances.sort_values(ascending=False))

feature_0    0.0
feature_1    0.0
feature_2    0.0
feature_3    0.0
feature_4    0.0
feature_5    0.0
feature_6    0.0
dtype: float64


In [ ]:
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))

[[1204020]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1204020

    accuracy                           1.00   1204020
   macro avg       1.00      1.00      1.00   1204020
weighted avg       1.00      1.00      1.00   1204020

ROC AUC Score: nan


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
np.unique(y_test, return_counts=True)

(array([0, 1]), array([1270881,    1643]))

In [ ]:
df = df.drop(columns=['type', 'nameOrig', 'nameDest'])

In [ ]:
X = df.drop(columns=['isFraud'])
y = df['isFraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
model = RandomForestClassifier()
model.fit(X_resampled, y_resampled)

RandomForestClassifier()

In [ ]:
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))

[[1269698    1183]
 [    125    1518]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270881
           1       0.56      0.92      0.70      1643

    accuracy                           1.00   1272524
   macro avg       0.78      0.96      0.85   1272524
weighted avg       1.00      1.00      1.00   1272524

ROC AUC Score: 0.9614944044143443


The model’s feature importance analysis highlights a few critical variables that contribute most to predicting fraud. These include:

Transaction amount: Larger-than-usual transaction values are often associated with fraud attempts.

Balance before and after transaction (origin and destination): Sudden or unusual changes in balances can indicate suspicious activity, such as accounts being drained or topped up irregularly.

Old balance destination: If the destination account previously had zero or negligible balance, but suddenly receives a large transfer, it could suggest fraudulent behavior.

These features stood out during model training as they consistently helped the classifier distinguish between normal and fraudulent transactions.



 factors are logical and align well with typical fraud patterns observed in real financial systems.

For instance, fraudsters often attempt to transfer unusually large amounts in a short time frame, especially to dormant or newly created accounts. Sudden changes in account balances, particularly when funds are moved rapidly without any prior pattern, are also strong indicators of fraudulent behavior. The model’s reliance on these variables suggests it has captured these behavioral cues effectively.

These features are consistent with real-world fraud detection practices used in financial institutions, which further validates their relevance and importance in the model.

While updating its infrastructure, the company should implement proactive fraud prevention mechanisms that combine both technical and procedural safeguards:

Real-time monitoring: Integrate fraud detection models into live transaction pipelines to flag or block suspicious activities instantly.

Anomaly detection systems: Use threshold-based or behavioral anomaly systems alongside ML models for layered defense.

Role-based access control: Ensure that sensitive financial systems are protected with strict permission hierarchies to prevent internal misuse.

Data encryption & secure APIs: Encrypt sensitive transaction data and ensure secure communication channels across services.

Audit trails: Maintain detailed logs of system activity and financial movements to support post-incident analysis.

Frequent model retraining: Regularly update fraud models with fresh data to adapt to evolving fraud patterns.

These measures create a robust infrastructure that is harder to exploit and quicker to respond to emerging fraud tactics.

To assess the effectiveness of these actions, the company should establish fraud performance metrics and monitor them consistently over time. Key indicators include:

Reduction in fraud rate: Compare the number and percentage of fraud cases detected before and after the implementation.

False positive rate: Ensure that genuine transactions are not being wrongly flagged or blocked.

Detection time: Measure how quickly the system identifies fraud after a transaction is initiated.

User complaints or reversals: A decrease in customer-reported frauds or refund requests is a strong indicator of system improvement.

Model performance: Track evaluation metrics like ROC AUC, precision, recall, and F1-score after periodic model retraining.

Additionally, conducting regular audits, penetration testing, and user behavior analysis can help confirm whether the system remains resilient and effective in a live environment.

